加载环境变量

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
# 设置用户代理
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

1. 加载文档

In [2]:
from langchain_community.document_loaders import WebBaseLoader # pip install langchain-community

loader = WebBaseLoader(
    web_paths=("https://zh.wikipedia.org/wiki/黑神话：悟空",)
)
docs = loader.load()

2. 文档分块

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

3. 设置嵌入模型

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings # pip install langchain-huggingface

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-zh-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)


4. 创建向量存储

In [5]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(all_splits)

['d35fe69b-a3c9-4182-9447-bba08ed4a2d5',
 'e70f0d35-1318-4269-b03e-450fd6dfef91',
 'fa818940-8365-4014-aaf2-07e904c173fd',
 '059ae77a-f766-40ab-9e2b-d1cd6d2f91a6',
 'c20bcc80-2dba-4e71-8282-10520a554687',
 '57a2f105-dc22-486a-8ae8-ce2079f54ff3',
 '7ed03060-72f0-4622-ad3a-17376f8f223e',
 'fe9e20a3-7e08-44f3-b2df-62430b36bf68',
 'a32ce940-70d2-4adb-afb2-0984b1c0bc0e',
 '6b9db456-59c4-42f8-8457-a185e8dbc291',
 'c9ee764a-8d39-454c-b57e-7eec5b2767bc',
 '48ac02a5-4e36-4150-b98a-f9b9937b47d9',
 'a48c84b9-8737-4dc5-a3e4-a94cff1b11c0',
 '0789f785-bd02-4470-8cb0-0b468742f22a',
 '3d233d9c-1eeb-42fe-a214-1726de5f6ebc',
 '736bcbd9-62e8-40b9-9548-929e6f28a050',
 '136557cd-1d38-432b-af22-32eb679af835',
 'abd8215d-bc11-4a07-b47c-814705c9d96f',
 '5c806d70-1eb1-45bf-be8b-b66915fef781',
 'cf2880ac-d803-4e2b-abf4-502fbcfb23c0',
 '1dc7e8ee-4847-48d3-87ab-97d82088656b',
 '4c84e94b-e8c4-43c3-8973-d53ebbad1736',
 '3fd73faf-389f-4824-9b54-35bb6a3447d1',
 'f643d791-303e-4ec0-a430-531d543364f7',
 '0c6ffe0f-a187-

5. 构建用户查询

In [6]:
question = "黑悟空有哪些游戏场景？"

6. 在向量存储中搜索相关文档，并准备上下文内容

In [7]:
retrieved_docs = vector_store.similarity_search(question, k=3)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

7. 构建提示模板

In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
                基于以下上下文，回答问题。如果上下文中没有相关信息，
                请说"我无法从提供的上下文中找到相关信息"。
                上下文: {context}
                问题: {question}
                回答:""")

8. 使用大语言模型生成答案

In [9]:
from langchain_deepseek import ChatDeepSeek # pip install langchain-deepseek

llm = ChatDeepSeek(
    model="deepseek-chat",  # DeepSeek API 支持的模型名称
    temperature=0.7,        # 控制输出的随机性
    max_tokens=2048,        # 最大输出长度
    api_key=os.getenv("DEEPSEEK_API_KEY")  # 从环境变量加载API key
)

9. 生成答案

In [10]:
print(f"问题: {question}")
print("-"*50)
answer = llm.invoke(prompt.format(question=question, context=docs_content))
print(f"回答: {answer.content}")

问题: 黑悟空有哪些游戏场景？
--------------------------------------------------
回答: 根据提供的上下文，《黑神话：悟空》的游戏场景融合了中国文化和自然地标，具体包括以下取景地：  
1. **重庆大足石刻**  
2. **山西省**的多个地点：  
   - 小西天  
   - 南禅寺  
   - 铁佛寺  
   - 广胜寺  
   - 鹳雀楼（游戏中的标志性地标之一）  
3. 其他未明确列举的中国古建场景（上下文提到共36个取景地，其中27个位于山西）。  

这些场景在游戏中通过三维建模还原，并融入了佛教、道教哲学元素，与《西游记》的神魔世界观相结合。
